In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
if True:
	df_migration = pd.read_csv('../Datasets/limbo/migration_breda.csv').drop(columns=['ID'], axis=1)
	df = df_migration.copy()
	df['Year'] = df['Year'].astype(str)

	expanded_dfs = []

	for index, row in df.iterrows():
		current_year = int(row['Year'])
		next_year = current_year + 1

		# divide by 12 to get monthly values
		yearly_value = row['Migrated']
		monthly_value = yearly_value

		# Interpolate values between years
		if next_year <= 2022:
			start_value = monthly_value
			end_value = df.loc[(df['NeighbourhoodCode'] == row['NeighbourhoodCode']) & (df['Year'] == str(next_year)), 'Migrated'].values
			if len(end_value) > 0:
				end_value = end_value[0]
			else:
				end_value = yearly_value

			interpolated_values = [
				math.floor(start_value + (end_value - start_value) * ((month - 1) / 12))
				for month in range(1, 13)
			]

			expanded_df = pd.DataFrame({
				'NeighbourhoodCode': [row['NeighbourhoodCode']] * 12,
				'Year': [str(current_year)] * 12,
				'Month': range(1, 13),
				'Migrated': interpolated_values
			})

			expanded_dfs.append(expanded_df)
		else:
			# Add static values for the last year
			interpolated_values = [math.floor(monthly_value)] * 12

			expanded_df = pd.DataFrame({
				'NeighbourhoodCode': [row['NeighbourhoodCode']] * 12,
				'Year': [str(current_year)] * 12,
				'Month': range(1, 13),
				'Migrated': interpolated_values
			})

			expanded_dfs.append(expanded_df)

	expanded_df = pd.concat(expanded_dfs, ignore_index=True)
	expanded_df.sort_values(['NeighbourhoodCode', 'Year', 'Month'], inplace=True)
	expanded_df.reset_index(drop=True, inplace=True)

	expanded_df.to_csv('../Datasets/limbo/migration_breda_expanded.csv', index=False)

In [ ]:
if True:
	df_livability = pd.read_csv('../Datasets/cleaned/livability_raw.csv')
	df = df_livability.copy()

	df['Year'] = df['Year'].astype(str)

	# List the columns to interpolate and expand
	columns_to_expand = ["lbm","afw","fys","onv","soc","vrz","won"]

	expanded_dfs = []

	for index, row in df.iterrows():
		current_year = int(row['Year'])
		next_year = current_year + 1

		# Interpolate values between years
		if next_year <= 2022:
			start_values = row[columns_to_expand]
			end_values = df.loc[(df['NeighbourhoodCode'] == row['NeighbourhoodCode']) & (df['Year'] == str(next_year)), columns_to_expand].values
			if len(end_values) > 0:
				end_values = end_values[0]
			else:
				end_values = start_values

			interpolated_values = []
			for i in range(len(columns_to_expand)):
				start_value = start_values[i]
				end_value = end_values[i]
				interpolated_value = [
					#math.floor(start_value + (end_value - start_value) * ((month - 1) / 12))
					round(start_value + (end_value - start_value) * ((month - 1) / 12), 5)
					for month in range(1, 13)
				]
				interpolated_values.append(interpolated_value)

			expanded_df = pd.DataFrame({
				'NeighbourhoodCode': [row['NeighbourhoodCode']] * 12,
				'Year': [str(current_year)] * 12,
				'Month': range(1, 13),
			})

			for i, column in enumerate(columns_to_expand):
				expanded_df[column] = interpolated_values[i]

			expanded_dfs.append(expanded_df)
		else:
			# Add static values for the last year
			static_values = row[columns_to_expand]

			expanded_df = pd.DataFrame({
				'NeighbourhoodCode': [row['NeighbourhoodCode']] * 12,
				'Year': [str(current_year)] * 12,
				'Month': range(1, 13),
			})

			for column in columns_to_expand:
				#expanded_df[column] = [math.floor(static_values[column])] * 12
				expanded_df[column] = [static_values[column]] * 12

			expanded_dfs.append(expanded_df)


	expanded_df = pd.concat(expanded_dfs, ignore_index=True)
	expanded_df.sort_values(['NeighbourhoodCode', 'Year', 'Month'], inplace=True)
	expanded_df.reset_index(drop=True, inplace=True)

	expanded_df.to_csv('../Datasets/limbo/livability_extended.csv', index=False)